W pierwszym kroku należy załadować obraz, zmienić go na grayscale, a potem zapisac do macierzy żeby zaczać wykonywać na nim operacje, oraz zaimportować wszystkie potrzebne biblioteki.

In [ ]:
from helpers import *
import matplotlib.pyplot as plt

In [ ]:
imgMatrix = load_image("img/Kropka.jpg")

Następny etap to sinogram.
W następnym bloku będą też ustalane stałe detektorów.

In [ ]:
alpha = 2
phi_range = 90
num_detectors = 180
num_iterations = 90

Transformata Radona.

In [ ]:
whole_sinogram = radon_transform(imgMatrix, alpha, phi_range, num_detectors, num_iterations)
plt.matshow(whole_sinogram, cmap=plt.cm.gray)
plt.show()

Poszczególne iteracje transformaty.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

int_range_sinogram = widgets.IntSlider(min=1, max=num_iterations)
output1 = widgets.Output()
display(int_range_sinogram, output1)

def on_value_change_sinogram(change):
    with output1:
        sinogram = radon_transform(imgMatrix, alpha, phi_range, num_detectors, num_iterations=change['new'])
        clear_output(wait=True)
        plt.matshow(sinogram, cmap=plt.cm.gray)
        plt.show()

int_range_sinogram.observe(on_value_change_sinogram, names='value')

Rekonstrukcja obrazu (backprojection).

In [ ]:
whole_reconstruction = backprojection(imgMatrix, whole_sinogram, alpha, phi_range, num_detectors, num_iterations)
plt.matshow(whole_reconstruction, cmap=plt.cm.gray)
plt.show()

Poszczególne iteracje backprojection.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

int_range_backprojection = widgets.IntSlider(min=1, max=num_iterations)
output2 = widgets.Output()
display(int_range_backprojection, output2)

def on_value_change_backprojection(change):
    with output2:
        recounstruction = backprojection(imgMatrix, whole_sinogram, alpha, phi_range, num_detectors, num_iterations=change['new'])
        clear_output(wait=True)
        plt.matshow(recounstruction, cmap=plt.cm.gray)
        plt.show()

int_range_backprojection.observe(on_value_change_backprojection, names='value')

Zapis do pliku dicom oraz wpisanie podstawowych danych pacjenta jak i ścieżki gdzie zapisać plik.

In [ ]:
patient_data = {"PatientName": "Sebastian",
               "PatientID": "123",
               "ImageComments": "lorem ipsum"}

save_path = "img-dicom/xyz.dcm"

In [ ]:
save_as_dicom(save_path, whole_reconstruction, patient_data)

Odczytywanie pliku dicom.

In [ ]:
read_from_dicom("img-dicom/xyz.dcm")